In [48]:
from mido import MidiFile

In [49]:
## Array con archivos cargados de test

files_mid = ['Concerto Grosso op n8 1-2mov','Sonata n5 op25 1mov','Sonata n5 op25 2mov', \
             'bach_inventions_772_[free]_(c)simonetto','Contradance n1','Contradance n3', \
             'K02 Minuet in F','K03 Allegro in Bb','K04 Minuet in F','Piano Concerto n1 2mov',\
             'racelegy','rach1-1','rach1-3','rach2-1', '6 Impromptus n1', '6 Impromptus n2'
            , '6 Impromptus n3', '6 Impromptus n4', '6 Impromptus n5', '6 Impromptus n6']

file_num = 1
files_mid[file_num]

'Sonata n5 op25 1mov'

In [50]:
## Levanta el archivo midi, devuelve cantidad de tracks y mensajes contenidos

mid = MidiFile(files_mid[file_num] + '.mid', clip=True)
print(mid)
print('ticks por beat:', mid.ticks_per_beat)
duracion_tema = mid.length
print('segundos:', duracion_tema)


<midi file 'Sonata n5 op25 1mov.mid' type 1, 8 tracks, 9180 messages>
ticks por beat: 120
segundos: 423.48499999998023


In [51]:
## Variables globales para el ploteo

plot_from_second = 0
plot_to_second = 60 #duracion_tema

type 0 (single track): all messages are saved in one track

type 1 (synchronous): all tracks start at the same time

type 2 (asynchronous): each track is independent of the others

In [52]:
## Importa la tabla que parsea numeros de notas a nombre y octava
import pandas as pd
midi_notes = pd.read_csv('midi_notes.csv',sep=";")
midi_notes

,note_number,Octave,note_name,note
0,0,0,C,C0
1,1,0,C#,C#0
2,2,0,D,D0
3,3,0,D#,D#0
4,4,0,E,E0
...,...,...,...,...
123,123,10,D#,D#10
124,124,10,E,E10
125,125,10,F,F10
126,126,10,F#,F#10


In [53]:
## Importa la tabla que contiene acordes y escalas
import pandas as pd
midi_scales_chords = pd.read_csv('scales.csv',sep=";")
midi_scales_chords

,note_name,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,U
0,C,1.0,7.0,NaN,6.0,NaN,5.0,NaN,4.0,3.0,NaN,2.0,NaN,1
1,C#,NaN,1.0,7.0,NaN,6.0,NaN,5.0,NaN,4.0,3.0,NaN,2.0,1
2,D,2.0,NaN,1.0,7.0,NaN,6.0,NaN,5.0,NaN,4.0,3.0,NaN,1
3,D#,NaN,2.0,NaN,1.0,7.0,NaN,6.0,NaN,5.0,NaN,4.0,3.0,1
4,E,3.0,NaN,2.0,NaN,1.0,7.0,NaN,6.0,NaN,5.0,NaN,4.0,1
5,F,4.0,3.0,NaN,2.0,NaN,1.0,7.0,NaN,6.0,NaN,5.0,NaN,1
6,F#,NaN,4.0,3.0,NaN,2.0,NaN,1.0,7.0,NaN,6.0,NaN,5.0,1
7,G,5.0,NaN,4.0,3.0,NaN,2.0,NaN,1.0,7.0,NaN,6.0,NaN,1
8,G#,NaN,5.0,NaN,4.0,3.0,NaN,2.0,NaN,1.0,7.0,NaN,6.0,1
9,A,6.0,NaN,5.0,NaN,4.0,3.0,NaN,2.0,NaN,1.0,7.0,NaN,1


In [54]:
## Itera sobre los mensaje midi de los tracks con mensjaes y genera un dataframe con notas, velocity, tick_start tick_stop

tema_df = pd.DataFrame(columns=['note_num', 'start', 'end', 'velocity','time_dif', 'track_name']) 

count_notes = 0
last_note_on = 0
info = []
controls = []
key_signatures = []
time_signatures = []
n_track = 0
n_tracks_used = 0
for track in mid.tracks:
    
    track_name = track.name + str(n_track)
    info.append(track_name)
    n_track = n_track + 1 
    if len(track) > 100:      
        n_tracks_used = n_tracks_used + 1
        print('mensajes:',len(track))
    time = 0
    has_note_off = any(msg.type == 'note_off' for msg in track)
    for msg in track:          
        if msg.type in ['note_on', 'note_off']:
            time = time + msg.time
            if (has_note_off and (msg.type == 'note_on')) or (not has_note_off and msg.velocity > 0):
                tema_df.loc[count_notes] = [msg.note, time,-1, msg.velocity, time - last_note_on, track_name]
                count_notes = count_notes + 1
                last_note_on = time
            else:
                tema_df.loc[(tema_df.note_num == msg.note) & (tema_df.end == -1),'end'] = time
        else:
            if (msg.type == 'control_change'):
                controls.append(msg.value)
            elif (msg.type == 'key_signature'):
                key_signatures.append(msg.key)
            elif (msg.type == 'time_signature'):
                time_signatures.append(str(msg.numerator) + '/' + str(msg.denominator))
                    
                    
                     

print('info tracks:', info)
print('tipo note off:', has_note_off)
controls = pd.Series(controls).head(40)
controls = controls[controls > 10].sum() / n_tracks_used
print('cantidad eventos pedal:', controls)
print('claves:', key_signatures)
print('time signatures:', time_signatures)
display(tema_df)  

mensajes: 9152
info tracks: ['Clementi Muzio,0', 'Clementi M.1', 'Sonata  Op.25 n.5 in f#2', 'Allegro con Espressione3', 'Sequence by4', 'Gabriele Giulimondi5', 'giulim@mclink.it6', 'Novemner 19967']
tipo note off: False
cantidad eventos pedal: 2644.0
claves: ['C']
time signatures: ['4/4']


,note_num,start,end,velocity,time_dif,track_name
0,78,0,68,55,0,Clementi M.1
1,77,62,105,83,62,Clementi M.1
2,78,94,125,75,32,Clementi M.1
3,80,121,160,84,27,Clementi M.1
4,78,148,191,83,27,Clementi M.1
...,...,...,...,...,...,...
3954,61,70543,70833,42,54,Clementi M.1
3955,49,70649,70667,31,106,Clementi M.1
3956,37,70650,70725,33,1,Clementi M.1
3957,42,70719,71128,29,69,Clementi M.1


In [55]:
## Calcula la duración de cada nota

tema_df.loc[:,'duration'] = tema_df.end - tema_df.start

In [56]:
## Parsea a enteros los valores numéricos

import numpy as np
for col in tema_df.loc[:,tema_df.columns!="track_name"].columns:
    tema_df[col] = pd.to_numeric(tema_df[col])

In [57]:
## agrega el nombre y octava de notas a la tabla

tema_df = pd.merge(tema_df, midi_notes,how='left',left_on='note_num',right_on='note_number').drop('note_number',axis=1)
tema_df

,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note
0,78,0,68,55,0,Clementi M.1,68,6,F#,F#6
1,77,62,105,83,62,Clementi M.1,43,6,F,F6
2,78,94,125,75,32,Clementi M.1,31,6,F#,F#6
3,80,121,160,84,27,Clementi M.1,39,6,G#,G#6
4,78,148,191,83,27,Clementi M.1,43,6,F#,F#6
...,...,...,...,...,...,...,...,...,...,...
3954,61,70543,70833,42,54,Clementi M.1,290,5,C#,C#5
3955,49,70649,70667,31,106,Clementi M.1,18,4,C#,C#4
3956,37,70650,70725,33,1,Clementi M.1,75,3,C#,C#3
3957,42,70719,71128,29,69,Clementi M.1,409,3,F#,F#3


In [58]:
tema_df.head(10)

,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note
0,78,0,68,55,0,Clementi M.1,68,6,F#,F#6
1,77,62,105,83,62,Clementi M.1,43,6,F,F6
2,78,94,125,75,32,Clementi M.1,31,6,F#,F#6
3,80,121,160,84,27,Clementi M.1,39,6,G#,G#6
4,78,148,191,83,27,Clementi M.1,43,6,F#,F#6
5,81,187,267,69,39,Clementi M.1,80,6,A,A6
6,73,257,273,53,70,Clementi M.1,16,6,C#,C#6
7,73,288,407,64,31,Clementi M.1,119,6,C#,C#6
8,69,358,372,34,70,Clementi M.1,14,5,A,A5
9,66,359,372,35,1,Clementi M.1,13,5,F#,F#5


In [59]:
## Reemplaza outlyers de duración

notes_weight = pd.cut(tema_df.duration, 6)
print('duration notes before:')
display(notes_weight.value_counts())

outlyeras_duration = pd.DataFrame(tema_df.duration.quantile([0.05,0.95]))

mask_outlyers_lower = tema_df.duration < outlyeras_duration.duration[0.05]
print('lowers modificados:',mask_outlyers_lower.sum(), 'val:', outlyeras_duration.duration[0.05])
tema_df.loc[mask_outlyers_lower,'duration'] = outlyeras_duration.duration[0.05]

mask_outlyers_higher = tema_df.duration > outlyeras_duration.duration[0.95]
print('highers modificados:', mask_outlyers_higher.sum(), 'val:', outlyeras_duration.duration[0.95])
tema_df.loc[mask_outlyers_higher,'duration'] = outlyeras_duration.duration[0.95]

notes_weight = pd.cut(tema_df.duration, 6)
print('duration notes after:')
display(notes_weight.value_counts().head(7))

duration notes before:


(0.575, 71.833]       3716
(71.833, 142.667]      170
(142.667, 213.5]        55
(284.333, 355.167]       8
(355.167, 426.0]         5
(213.5, 284.333]         5
Name: duration, dtype: int64

lowers modificados: 160 val: 8.0
highers modificados: 197 val: 78.0
duration notes after:


(7.93, 19.667]      1657
(19.667, 31.333]     998
(31.333, 43.0]       511
(66.333, 78.0]       310
(43.0, 54.667]       259
(54.667, 66.333]     224
Name: duration, dtype: int64

In [60]:
## Categoriza duración

cat_duration = pd.cut(tema_df.duration, 6, labels=['xxs','xs','s','m','l','xl'])
print('category duration notes:')
display(cat_duration.value_counts())
tema_df.loc[:,'cat_duration'] = cat_duration

category duration notes:


xxs    1657
xs      998
s       511
xl      310
m       259
l       224
Name: duration, dtype: int64

In [61]:
## Categoriza dudación

cat_velocity = pd.cut(tema_df.velocity, 6, labels=['pp','p','m','mf','f','ff'])
print('category velocity notes:')
display(cat_velocity.value_counts())
tema_df.loc[:,'cat_velocity'] = cat_velocity

category velocity notes:


mf    1267
f     1104
m      917
p      394
ff     238
pp      39
Name: velocity, dtype: int64

In [62]:
## Describe, muchos de estos valores van a ser utiles como predictores

tema_describe = tema_df.describe()

In [63]:
## Cuantificación de notas:
## define cuales notas están tan cera que deberían estar en el mismo instante

unit_duplicate = tema_describe.loc['min','duration'] / 1.5
tema_df.loc[:,'note_simultaneous'] = tema_df.time_dif < unit_duplicate
print('ticks para quantizar: ',unit_duplicate)

ticks para quantizar:  5.333333333333333


In [64]:
## Reemplaza los tiempos de notas muy cercanas por notas simultaneas

tema_df2_mask = (tema_df.iloc[1:len(tema_df),:].note_simultaneous == True)
tema_df2_mask = tema_df2_mask
tema_df2_mask[0] = False

tema_df.loc[tema_df2_mask, 'start'] = np.NaN
tema_df.start = tema_df.start.fillna(method='ffill')
tema_df.start = tema_df.start.astype(int)
tema_df

,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note,cat_duration,cat_velocity,note_simultaneous
0,78,0,68,55,0,Clementi M.1,68.0,6,F#,F#6,xl,m,True
1,77,62,105,83,62,Clementi M.1,43.0,6,F,F6,s,f,False
2,78,94,125,75,32,Clementi M.1,31.0,6,F#,F#6,xs,f,False
3,80,121,160,84,27,Clementi M.1,39.0,6,G#,G#6,s,f,False
4,78,148,191,83,27,Clementi M.1,43.0,6,F#,F#6,s,f,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3954,61,70543,70833,42,54,Clementi M.1,78.0,5,C#,C#5,xl,m,False
3955,49,70649,70667,31,106,Clementi M.1,18.0,4,C#,C#4,xxs,p,False
3956,37,70649,70725,33,1,Clementi M.1,75.0,3,C#,C#3,xl,p,True
3957,42,70719,71128,29,69,Clementi M.1,78.0,3,F#,F#3,xl,p,False


In [65]:
## calcula la cantidad de ticks x segundo

ticks_por_seg = tema_df.end.max() / duracion_tema
print('Ticks por segundo:', ticks_por_seg)

Ticks por segundo: 167.95872344948066


In [66]:
## Calcula la cantidad de notas que existen en simultaneo

tema_df_simultaneous =  tema_df.start.value_counts()
tema_df_simultaneous_times = tema_df_simultaneous.loc[tema_df_simultaneous > 1].index.to_list()
print('cant. notas en simultaneo:',len(tema_df_simultaneous_times))

tema_df.note_simultaneous = False
tema_df.loc[:,'note_simultaneous'] = tema_df.start.apply(lambda x: True if x in tema_df_simultaneous_times else False) 

cant. notas en simultaneo: 836


In [67]:
## Cada nota l y x l van a tener mas puntos que despues van a ser removidos

tema_df.loc[:,'note_fake'] = 0
long_notes_mask = tema_df.cat_duration.apply(lambda x: x in ['l','xl'])
fake_notes = tema_df[long_notes_mask]
fake_notes.start = fake_notes.apply(lambda x: x.start + (x.end - x.start) / 2 if x.cat_duration == 'l' else x.start + (x.end - x.start) / 3,axis=1)

C:\Users\Patricio\anaconda3\envs\dhdsblend\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [68]:
fake_notes.duration = fake_notes.duration / 2
fake_notes.note_fake = 1

xlong_notes_mask = tema_df.cat_duration == 'xl'
fake_notes_lx = tema_df[xlong_notes_mask]
fake_notes_lx.start = fake_notes_lx.start + (fake_notes_lx.end - fake_notes_lx.start)  / 3 * 2
fake_notes_lx.duration = fake_notes_lx.duration / 2
fake_notes_lx.note_fake = 1


tema_df = pd.concat([tema_df,fake_notes, fake_notes_lx]).sort_values('start')

In [69]:
## Convierte unidad de medida de timpo Ticks a segundos en cada nota

tema_df.loc[:,'segundo'] = tema_df.start / ticks_por_seg

In [70]:
## Implementacion DBSCAN para encontrar grupos de notas y segmentarlas

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

X = tema_df[['start', 'note_num']]
X.loc[:,'start'] = X.start / 15
X.loc[:,'note_num'] = X.note_num / 0.2

X = X.values
print(X[0:5])

# Implementación de DBSCAN

dbscan = DBSCAN( eps=29, min_samples=3)
y_dbscan = dbscan.fit_predict(X)
y_dbscan

labels = dbscan.labels_
#unique_labels=set(labels);
tema_df.loc[:,'grupos'] = labels
cant_grupos = pd.Series(labels).unique().shape[0]
print('cantidad de grupos:',cant_grupos)

[[  0.         390.        ]
 [  1.51111111 390.        ]
 [  3.02222222 390.        ]
 [  4.13333333 385.        ]
 [  6.26666667 390.        ]]
cantidad de grupos: 31


C:\Users\Patricio\anaconda3\envs\dhdsblend\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



import matplotlib.pyplot as plt
# Lista de los puntos core y máscara para graficar
core_samples_mask = np.zeros_like(dbscan.labels_, dtype=bool)
core_samples_mask[dbscan.core_sample_indices_] = True

fig, ax = plt.subplots(figsize=(80,80))

colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

for i,l in enumerate(unique_labels):
    if l ==-1:
        ax.scatter(X[labels==l,0], X[labels==l,1],c =colors[i], edgecolor='black', marker='o',s=40,label='Noise')
    else:
        ax.scatter(X[labels==l,0], X[labels==l,1],c =colors[i], edgecolor='blue', marker='o',s=40,label='Cluster '+str(l))
    
ax.legend()
ax.set_title('DBSCAN clustering');

In [71]:
## Concatena los grupos creados con DBSCAN con las cadenas de notas recibidas en cada track

tema_df.loc[:,'grupos_track'] = tema_df.grupos.astype(str) + "_" + tema_df.track_name
tema_df

,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note,cat_duration,cat_velocity,note_simultaneous,note_fake,segundo,grupos,grupos_track
0,78,0.000000,68,55,0,Clementi M.1,68.0,6,F#,F#6,xl,m,False,0,0.000000,0,0_Clementi M.1
0,78,22.666667,68,55,0,Clementi M.1,34.0,6,F#,F#6,xl,m,False,1,0.134954,0,0_Clementi M.1
0,78,45.333333,68,55,0,Clementi M.1,34.0,6,F#,F#6,xl,m,False,1,0.269908,0,0_Clementi M.1
1,77,62.000000,105,83,62,Clementi M.1,43.0,6,F,F6,s,f,False,0,0.369138,0,0_Clementi M.1
2,78,94.000000,125,75,32,Clementi M.1,31.0,6,F#,F#6,xs,f,False,0,0.559661,0,0_Clementi M.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3954,61,70736.333333,70833,42,54,Clementi M.1,39.0,5,C#,C#5,xl,m,False,1,421.153078,9,9_Clementi M.1
3958,49,70840.000000,71082,15,3,Clementi M.1,39.0,4,C#,C#4,xl,pp,True,1,421.770293,9,9_Clementi M.1
3957,42,70855.333333,71128,29,69,Clementi M.1,39.0,3,F#,F#3,xl,p,True,1,421.861585,29,29_Clementi M.1
3958,49,70961.000000,71082,15,3,Clementi M.1,39.0,4,C#,C#4,xl,pp,True,1,422.490708,9,9_Clementi M.1


In [112]:
# Crea una mascara para limitar el ploteo en el tiempo
plot_from_second = 0
plot_to_second = 100

mask_tiempo_from = (tema_df.segundo >= plot_from_second)
mask_tiempo_to =  (tema_df.segundo <= plot_to_second)

size_plot = ['duration', 'velocity'][1]
color_plot = ['grupos_track', 'velocity','note_simultaneous', 'cat_velocity'][2]
print(size_plot,color_plot)


velocity note_simultaneous


In [114]:
## Poltea las notas en el tiempo
import plotly.express as px

fig = px.scatter(data_frame = tema_df[mask_tiempo_from & mask_tiempo_to], x = "segundo", y = "note_num",
                 size = size_plot, color = color_plot, opacity = 1, color_discrete_sequence=None, labels='note', height=600,)

fig.show()

In [27]:
notes_fake_mask = tema_df.note_fake == 1
tema_df.drop(tema_df[notes_fake_mask].index, inplace=True)

In [28]:
## Imprime el dataset completo ordenado por grupos descartando el ruido
## Calcula el ruido
print('noise notes: %', round((tema_df.grupos == -1).sum() / tema_df.shape[0],2))
mask_noise = tema_df.grupos != -1
tema_df.sort_values(['grupos_track','start'],ascending=[True,True],inplace=True)
tema_df[mask_noise].head(10)

noise notes: % 0.0


C:\Users\Patricio\anaconda3\envs\dhdsblend\lib\site-packages\ipykernel_launcher.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note,cat_duration,cat_velocity,note_simultaneous,note_fake,segundo,grupos,grupos_track
1,77,62.0,105,83,62,Clementi M.1,43.0,6,F,F6,s,f,False,0,0.369138,0,0_Clementi M.1
2,78,94.0,125,75,32,Clementi M.1,31.0,6,F#,F#6,xs,f,False,0,0.559661,0,0_Clementi M.1
3,80,121.0,160,84,27,Clementi M.1,39.0,6,G#,G#6,s,f,False,0,0.720415,0,0_Clementi M.1
4,78,148.0,191,83,27,Clementi M.1,43.0,6,F#,F#6,s,f,False,0,0.881169,0,0_Clementi M.1
6,73,257.0,273,53,70,Clementi M.1,16.0,6,C#,C#6,xxs,m,False,0,1.530138,0,0_Clementi M.1
8,69,358.0,372,34,70,Clementi M.1,14.0,5,A,A5,xxs,p,True,0,2.131476,0,0_Clementi M.1
9,66,358.0,372,35,1,Clementi M.1,13.0,5,F#,F#5,xxs,p,True,0,2.131476,0,0_Clementi M.1
13,84,575.0,612,91,67,Clementi M.1,37.0,7,C,C7,s,f,False,0,3.423460,0,0_Clementi M.1
14,85,601.0,640,85,26,Clementi M.1,39.0,7,C#,C#7,s,f,False,0,3.578260,0,0_Clementi M.1
15,86,634.0,667,93,33,Clementi M.1,33.0,7,D,D7,s,ff,False,0,3.774737,0,0_Clementi M.1


In [29]:
tema_df[mask_noise]

C:\Users\Patricio\anaconda3\envs\dhdsblend\lib\site-packages\ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,note_num,start,end,velocity,time_dif,track_name,duration,Octave,note_name,note,cat_duration,cat_velocity,note_simultaneous,note_fake,segundo,grupos,grupos_track
1,77,62.0,105,83,62,Clementi M.1,43.0,6,F,F6,s,f,False,0,0.369138,0,0_Clementi M.1
2,78,94.0,125,75,32,Clementi M.1,31.0,6,F#,F#6,xs,f,False,0,0.559661,0,0_Clementi M.1
3,80,121.0,160,84,27,Clementi M.1,39.0,6,G#,G#6,s,f,False,0,0.720415,0,0_Clementi M.1
4,78,148.0,191,83,27,Clementi M.1,43.0,6,F#,F#6,s,f,False,0,0.881169,0,0_Clementi M.1
6,73,257.0,273,53,70,Clementi M.1,16.0,6,C#,C#6,xxs,m,False,0,1.530138,0,0_Clementi M.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3947,68,70379.0,70394,54,15,Clementi M.1,15.0,5,G#,G#5,xxs,m,False,0,419.025571,9,9_Clementi M.1
3950,65,70408.0,70448,48,3,Clementi M.1,37.0,5,F,F5,s,m,True,0,419.198232,9,9_Clementi M.1
3951,66,70447.0,70481,44,36,Clementi M.1,34.0,5,F#,F#5,s,m,False,0,419.430432,9,9_Clementi M.1
3952,68,70485.0,70488,49,38,Clementi M.1,8.0,5,G#,G#5,xxs,m,True,0,419.656678,9,9_Clementi M.1


cat_results_array = []
last_note = 0
for index, row in data_cluster.iterrows():
    if index = 0:
        last_note = row.note_num
    row.
    cat_results_array.append(cat_result)

In [30]:
## Shape final del dataset

notas_totales = tema_df.shape[0]
print('total notas:',notas_totales)

total notas: 3425


In [31]:
## indice de actividad (cantidad de notas) por tiempo
cant_eventos_individuales = (notas_totales - len(tema_df_simultaneous_times) / 2)
actividad_por_tiempo = cant_eventos_individuales / duracion_tema
print(round(cant_eventos_individuales,0),'/',round(duracion_tema,0))
print('densidad (notas por segundo):', round(actividad_por_tiempo,2))

3007.0 / 423.0
densidad (notas por segundo): 7.1


In [32]:
print('categoría velocity:')
velocity_avg = tema_df.cat_velocity.value_counts(normalize=True)
display(velocity_avg)
print('categoría duration:')
length_notes_avg = tema_df.cat_duration.value_counts(normalize=True)
display(length_notes_avg)
print('groups:') 
display(tema_df.grupos_track.value_counts())

categoría velocity:


mf    0.326423
f     0.282628
m     0.223066
p     0.097226
ff    0.059854
pp    0.010803
Name: cat_velocity, dtype: float64

categoría duration:


xxs    0.483796
xs     0.291387
s      0.149197
m      0.075620
xl     0.000000
l      0.000000
Name: cat_duration, dtype: float64

groups:


9_Clementi M.1     2556
3_Clementi M.1      625
0_Clementi M.1      106
-1_Clementi M.1      15
11_Clementi M.1      13
15_Clementi M.1      12
23_Clementi M.1      12
4_Clementi M.1       12
24_Clementi M.1       7
13_Clementi M.1       7
16_Clementi M.1       7
27_Clementi M.1       6
28_Clementi M.1       6
26_Clementi M.1       5
17_Clementi M.1       5
18_Clementi M.1       5
25_Clementi M.1       5
19_Clementi M.1       4
20_Clementi M.1       4
5_Clementi M.1        4
12_Clementi M.1       3
1_Clementi M.1        3
7_Clementi M.1        1
22_Clementi M.1       1
6_Clementi M.1        1
Name: grupos_track, dtype: int64

In [33]:
tema_describe[['note_num','Octave','start','duration']].T

,count,mean,std,min,25%,50%,75%,max
note_num,3959.0,66.619096,11.081325,30.0,58.0,68.0,75.0,88.0
Octave,3959.0,5.086133,0.942461,2.0,4.0,5.0,6.0,7.0
start,3959.0,36010.954534,20228.719830,0.0,19039.0,36621.0,53644.5,70722.0
duration,3959.0,29.088154,19.458149,8.0,14.0,23.0,38.0,78.0


In [34]:
## Analiza proporciones de notas y duraciones mas repetidas

notes_weight = round(tema_df.note.value_counts(normalize=True) * 100,2)
print('notes most used:')
display(notes_weight.head(5))

notes_weight = round(tema_df.note_name.value_counts(normalize=True) * 100,2)
print('scale most used:')
all_values_notes = pd.DataFrame(notes_weight).reset_index()
most_probable_scale = all_values_notes.head(7)
display(most_probable_scale)
scale_coverage = notes_weight.head(7).sum()
print('covarage with scale:', scale_coverage)

notes most used:


A5     6.42
F#6    5.96
B5     5.84
D6     5.49
C#6    5.43
Name: note, dtype: float64

scale most used:


,index,note_name
0,F#,16.09
1,A,15.36
2,C#,14.48
3,G#,11.85
4,D,10.63
5,B,9.52
6,E,7.53


covarage with scale: 85.46


In [35]:
avr_vertical_notes = tema_df.note_simultaneous.sum() / notas_totales
print('avr. notas simultaneas:', avr_vertical_notes)

avr. notas simultaneas: 0.49635036496350365


In [36]:
cant_pedal_sustain = controls
cant_eventos_por_pedal = cant_eventos_individuales / cant_pedal_sustain if cant_pedal_sustain > 5 else np.NaN
cant_pedales_seg = cant_pedal_sustain / duracion_tema if cant_pedal_sustain > 5 else np.NaN
print('cantidad pedales:', cant_pedal_sustain)
print('cantidad eventos por pedal:', cant_eventos_por_pedal)
print('cantidad pedales/seg :', cant_pedales_seg)

cantidad pedales: 2644.0
cantidad eventos por pedal: 1.1372919818456884
cantidad pedales/seg : 6.24343247104413


In [37]:
#cantidad notas por octava
nombre_escala = pd.merge(most_probable_scale, midi_scales_chords, how='left', left_on='index', right_on='note_name')
nombre_escala.fillna(0,inplace=True)
nombre_escala_T = nombre_escala.T
nombre_escala_T
mask = nombre_escala_T.apply(lambda x: True if all(x != 0) else False, axis=1)
mask
tabla_esacla = nombre_escala_T[mask].T

tabla_esacla

,index,note_name_x,note_name_y,A,U
0,F#,16.09,F#,6,1
1,A,15.36,A,1,1
2,C#,14.48,C#,3,1
3,G#,11.85,G#,7,1
4,D,10.63,D,4,1
5,B,9.52,B,2,1
6,E,7.53,E,5,1


In [38]:
nombre_columna_Tmaj = tabla_esacla.columns[3]
tonalidad  = 0
tonalidad_escala = 'M'
dic = {6:1, 7:2, 1:3, 2:4, 3:5, 4:6, 5:7}
if nombre_columna_Tmaj != "U":
    tabla_esacla.set_index(tabla_esacla.columns[3],inplace=True,drop=False)
    mayor_chord_coverage = tabla_esacla.loc[[1,3,5],:'note_name_x'].sum()[1]
    minor_chord_coverage = tabla_esacla.loc[[6,1,3],:'note_name_x'].sum()[1]
    print('mayor chord:',round(mayor_chord_coverage,2),'| minor chord:',round(minor_chord_coverage,2))
    

    if minor_chord_coverage > mayor_chord_coverage:
        tonalidad = tabla_esacla.iloc[0,2]
        # reemplazo el 6to grado por la tónica
         # define desired replacements here

        tabla_esacla[nombre_columna_Tmaj] = tabla_esacla[nombre_columna_Tmaj].apply(lambda x: dic[x])
        tabla_esacla
        tonalidad_escala = 'm'
        tabla_esacla.set_index(nombre_columna_Tmaj,inplace=True,drop=False)
        tabla_esacla.sort_index(inplace=True)
    else:
        tonalidad = nombre_columna_Tmaj

    print(tonalidad + tonalidad_escala) 
else:
    if 'b' in key_signatures[0]:
        dict_keys = {'Db':'C#', 'Eb':'D#', 'Gb':'F#', 'Ab':'G#', 'Bb':'A#'}
        tonalidad =  dict_keys[key_signatures[0]]
    else:
        tonalidad = key_signatures[0]

    midi_scales_chords_weighted = pd.merge(midi_scales_chords[['note_name', tonalidad]], all_values_notes, how='left', left_on='note_name', right_on='index',)
    midi_scales_chords_weighted.drop('index',axis=1,inplace=True)
    midi_scales_chords_weighted.columns = ['note_name', 'scale', 'weight']
    midi_scales_chords_weighted.set_index(midi_scales_chords_weighted.columns[1],inplace=True,drop=False)
    mayor_chord_coverage = midi_scales_chords_weighted.loc[[1,3,5],'weight'].sum()
    minor_chord_coverage = midi_scales_chords_weighted.loc[[6,1,3],'weight'].sum()
    print('mayor chord:',round(mayor_chord_coverage,2),'| minor chord:',round(minor_chord_coverage,2))
    if minor_chord_coverage > mayor_chord_coverage:
        tonalidad_escala = 'm'
        midi_scales_chords_weighted_mask = midi_scales_chords_weighted.scale > 0
        midi_scales_chords_weighted.loc[midi_scales_chords_weighted_mask, 'scale'] = midi_scales_chords_weighted.loc[midi_scales_chords_weighted_mask, 'scale'].apply(lambda x: dic[x])
    midi_scales_chords_weighted.sort_index(inplace=True)
    midi_scales_chords_weighted.drop('scale',axis=1,inplace=True)
    scale_coverage = midi_scales_chords_weighted.head(7).sum()[1]
    tabla_esacla = midi_scales_chords_weighted
    tabla_esacla

mayor chord: 37.37 | minor chord: 45.93
F#m


In [39]:
music_stats = pd.DataFrame(columns=['tonalidad','tonalidad_escala','scale_coverage','mayor_chord_coverage','minor_chord_coverage',
                              'sc_1','sc_2','sc_3','sc_4','sc_5','sc_6','sc_7', 'avr_vertical_notes','cant_eventos_por_pedal',
                              'cant_pedales_seg','duracion_seg','tracks_used','cant_grupos', 'info_tracks', 'midi_info'])
music_stats.loc[0] = [tonalidad, tonalidad_escala, scale_coverage,  mayor_chord_coverage, minor_chord_coverage, 
               tabla_esacla.iloc[0,1], tabla_esacla.iloc[1,1], tabla_esacla.iloc[2,1], 
               tabla_esacla.iloc[3,1], tabla_esacla.iloc[4,1], tabla_esacla.iloc[5,1], 
               tabla_esacla.iloc[6,1], avr_vertical_notes,cant_eventos_por_pedal,
               cant_pedales_seg,duracion_tema,n_tracks_used, cant_grupos, " ".join(info), mid]


In [40]:
data_describe = pd.DataFrame(tema_describe.loc[tema_describe.index != 'count',['note_num','Octave','start','duration']].unstack())
data_describe.reset_index(inplace=True)
data_describe.loc[:,'name'] = data_describe.level_0 + "_" + data_describe.level_1
data_describe.set_index('name',inplace=True)
data_describe.drop(['level_0','level_1'],axis=1, inplace=True)

In [41]:
pd.concat([music_stats.T,data_describe,velocity_avg,length_notes_avg])

,0
tonalidad,F#
tonalidad_escala,m
scale_coverage,85.46
mayor_chord_coverage,37.37
minor_chord_coverage,45.93
sc_1,16.09
sc_2,11.85
sc_3,15.36
sc_4,9.52
sc_5,14.48


In [42]:
key_signatures

['C']

In [43]:
time_signatures

['4/4']

In [44]:
#separar eventos cada x tiempo
#conseguir dividir temas en partes (o no)

In [ ]:
import plotly.express as px
df = px.data.wind()
fig = px.bar_polar(df, r="frequency", theta="direction", color="strength", template="plotly_dark",
            color_discrete_sequence= px.colors.sequential.Plasma_r)
fig.show()